In [ ]:
import numpy as np
import pandas as pd
import random
import argparse
from utils import wass_dist_computation, convert_score

# Creation of a synthetic example

In [ ]:
def create_synthetic_attention_data_varying_lengths(num_samples, target_range, source_range):
    data = []
    for _ in range(num_samples):
        target_len = random.randint(*target_range)
        source_len = random.randint(*source_range)
        attn_matrix = np.random.rand(target_len, source_len)
        attn_matrix /= attn_matrix.sum()
        mt_len = target_len
        data.append({"idx": len(data), "attn": attn_matrix, "mt_len": mt_len})
    return pd.DataFrame(data)

dataset_stats = create_synthetic_attention_data_varying_lengths(1000, (10, 30), (10, 30))
dataset_reference_stats = create_synthetic_attention_data_varying_lengths(100000, (10, 30), (10, 30))

# Save the datasets to files
dataset_stats.to_pickle("dataset_stats_varying_lengths.pkl")
dataset_reference_stats.to_pickle("dataset_reference_stats_varying_lengths.pkl")

In [ ]:
args = argparse.Namespace(
    dataset_stats_path="dataset_stats_varying_lengths.pkl",
    dataset_reference_stats_path="dataset_reference_stats_varying_lengths.pkl",
    metric_space="uni",
    length_window=10.0,
    compare_to_uni=True,
    no_samples=100,
    bottom_k=4,
    seed=38
)

dataset_stats = pd.read_pickle(args.dataset_stats_path)
dataset_reference_stats = pd.read_pickle(args.dataset_reference_stats_path)

wasserstein_distances = wass_dist_computation(dataset_stats, dataset_reference_stats, args)
# for wass-to-data (compare_to_uni=False), the shape of wasserstein_distances should be [len(dataset_stats), no_samples]; you can aggregate via bottom-k
# e.g., wasserstein_distances = [np.mean(sorted(x_samp)[:args.bottom_k]) for x_samp in wasserstein_distances]

# for wass-to-uni (compare_to_uni=True), the shape of wasserstein_distances should be [len(dataset_stats)]. No need to aggregate.

# Replicate our results

In [ ]:
dataset_stats = pd.read_pickle("/home/nunomg/ot-hallucination-detection/hallucinations_deen_w_stats_and_scores.pkl")
dataset_attn_reference = pd.read_pickle("/home/nunomg/ot-hallucination-detection/reference_set_deen.pkl")
dataset_attn_reference = dataset_attn_reference.loc[~dataset_attn_reference.src.isin(dataset_stats.src)] # remove samples from the reference set that are in the test set

In [ ]:
# Replicate Wass-to-Data
args = argparse.Namespace(
    dataset_stats_path="dataset_stats_varying_lengths.pkl",
    dataset_reference_stats_path="dataset_reference_stats_varying_lengths.pkl",
    metric_space="l1",
    length_window=10.0,
    compare_to_uni=False,
    no_samples=1000,
    bottom_k=4,
    seed=38
)

# Replicate Wass-to-Uni
# args = argparse.Namespace(
#     dataset_stats_path="dataset_stats_varying_lengths.pkl",
#     dataset_reference_stats_path="dataset_reference_stats_varying_lengths.pkl",
#     metric_space="uni",
#     length_window=10.0,
#     compare_to_uni=True,
#     no_samples=1000,
#     bottom_k=4,
#     seed=38
# )    

In [ ]:
# Compute the Wasserstein distances
wasserstein_distances = wass_dist_computation(dataset_stats, dataset_attn_reference, args)
# aggregate the wasserstein distances via bottom-k
if not args.compare_to_uni:
    wass_dist_sorted = [sorted(x_samp) for x_samp in wasserstein_distances]
    wass_dist_agg = [np.mean(x_samp[:args.bottom_k]) for x_samp in wass_dist_sorted]
    dataset_stats["wass_to_data"] = wass_dist_agg
else:
    dataset_stats["wass_to_data"] = wasserstein_distances

In [ ]:
# compute metrics 
from utils import compute_detection_metrics
# compute detection metrics; you can also pass a list of metrics, e.g., ["alti", "cometkiwi"] -- just make sure to have the right sign for computation of AUROC and FPR@90TPR
dataset_stats["labse"] = -dataset_stats["labse"]
auroc_metrics = compute_detection_metrics(dataset_stats=dataset_stats, args=args, bottom_k=args.bottom_k, metrics=["labse"]) 
auroc_metrics

In [ ]:
# obtain wass_combo
dataset_stats_w_wass = pd.read_pickle("data_heldout_for_thresholding.pkl")
uni_val_threshold = dataset_stats_w_wass.sort_values(by="wass_to_unif", ascending=False)[:round(.001 * len(dataset_stats_w_wass))].wass_to_unif.values[-1]
final_score = []

max_val_wass_dist_mt = np.max(dataset_stats_w_wass.wass_to_data)
min_val_wass_dist_mt = np.min(dataset_stats_w_wass.wass_to_data)
max_val_wass_dist_uni = np.max(dataset_stats_w_wass.wass_to_unif)
min_val_wass_dist_uni = np.min(dataset_stats_w_wass.wass_to_unif)
for index in dataset_stats.index:
    sample = dataset_stats.loc[index]
    sample_len = len(sample.mt_ids)
    if sample.wass_to_unif >= uni_val_threshold:
        final_score_z = convert_score(sample.wass_to_unif, scores_old_min=min_val_wass_dist_uni, scores_old_max=max_val_wass_dist_uni, scores_new_min=min_val_wass_dist_mt, scores_new_max=max_val_wass_dist_mt)
        final_score.append(final_score_z)
    else:
        final_score.append(sample.wass_to_data)
dataset_stats["wass_combo"] = final_score

In [ ]:
auroc_metrics = compute_detection_metrics(dataset_stats=dataset_stats, args=args, bottom_k=args.bottom_k, metrics=["wass_to_data"]) 
auroc_metrics